# Unigram tokenization

The Unigram algorithm is often used in SentencePiece, which is the tokenization algorithm used by models like AlBERT, T5, mBART, Big Bird, and XLNet.

In [1]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [3]:
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
tokenizer

XLNetTokenizerFast(name_or_path='xlnet-base-cased', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<sep>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>', 'additional_special_tokens': ['<eop>', '<eod>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sep>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<pad>", 

In [4]:
# Like for BPE and WordPiece, we begin by counting the number of occurrences of each word in the corpus:

from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'▁This': 3,
             '▁is': 2,
             '▁the': 1,
             '▁Hugging': 1,
             '▁Face': 1,
             '▁Course.': 1,
             '▁chapter': 1,
             '▁about': 1,
             '▁tokenization.': 1,
             '▁section': 1,
             '▁shows': 1,
             '▁several': 1,
             '▁tokenizer': 1,
             '▁algorithms.': 1,
             '▁Hopefully,': 1,
             '▁you': 1,
             '▁will': 1,
             '▁be': 1,
             '▁able': 1,
             '▁to': 1,
             '▁understand': 1,
             '▁how': 1,
             '▁they': 1,
             '▁are': 1,
             '▁trained': 1,
             '▁and': 1,
             '▁generate': 1,
             '▁tokens.': 1})

Then, we need to initialize our vocabulary to something larger than the vocab size we will want at the end. We have to include all the basic characters (otherwise we won’t be able to tokenize every word), but for the bigger substrings we’ll only keep the most common ones, so we sort them by frequency:

In [5]:
char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # Loop through the subwords of length at least 2
        for j in range(i + 2, len(word) + 1):
            subwords_freqs[word[i:j]] += freq

# Sort subwords by frequency
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_subwords[:10]

[('▁t', 7),
 ('is', 5),
 ('er', 5),
 ('▁a', 5),
 ('▁to', 4),
 ('to', 4),
 ('en', 4),
 ('▁T', 3),
 ('▁Th', 3),
 ('▁Thi', 3)]

In [6]:
# We group the characters with the best subwords to arrive at an initial vocabulary of size 300:

token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}
token_freqs

{'▁': 31,
 'T': 3,
 'h': 9,
 'i': 13,
 's': 13,
 't': 14,
 'e': 21,
 'H': 2,
 'u': 6,
 'g': 5,
 'n': 11,
 'F': 1,
 'a': 12,
 'c': 3,
 'C': 1,
 'o': 13,
 'r': 9,
 '.': 4,
 'p': 2,
 'b': 3,
 'k': 3,
 'z': 2,
 'w': 3,
 'v': 1,
 'l': 7,
 'm': 1,
 'f': 1,
 'y': 3,
 ',': 1,
 'd': 4,
 '▁t': 7,
 'is': 5,
 'er': 5,
 '▁a': 5,
 '▁to': 4,
 'to': 4,
 'en': 4,
 '▁T': 3,
 '▁Th': 3,
 '▁Thi': 3,
 '▁This': 3,
 'Th': 3,
 'Thi': 3,
 'This': 3,
 'hi': 3,
 'his': 3,
 'th': 3,
 'ou': 3,
 'se': 3,
 '▁tok': 3,
 '▁toke': 3,
 '▁token': 3,
 'tok': 3,
 'toke': 3,
 'token': 3,
 'ok': 3,
 'oke': 3,
 'oken': 3,
 'ke': 3,
 'ken': 3,
 '▁s': 3,
 'ra': 3,
 'nd': 3,
 '▁i': 2,
 '▁is': 2,
 '▁th': 2,
 '▁the': 2,
 'the': 2,
 'he': 2,
 '▁H': 2,
 'in': 2,
 'rs': 2,
 'te': 2,
 '▁ab': 2,
 'ab': 2,
 '▁tokeni': 2,
 '▁tokeniz': 2,
 'tokeni': 2,
 'tokeniz': 2,
 'okeni': 2,
 'okeniz': 2,
 'keni': 2,
 'keniz': 2,
 'eni': 2,
 'eniz': 2,
 'ni': 2,
 'niz': 2,
 'iz': 2,
 'at': 2,
 'ti': 2,
 'tio': 2,
 'tion': 2,
 'io': 2,
 'ion': 2,
 'on':

Next, we compute the sum of all frequencies, to convert the frequencies into probabilities. For our model we will store the logarithms of the probabilities, because it’s more numerically stable to add logarithms than to multiply small numbers, and this will simplify the computation of the loss of the model:

In [8]:
from math import log

total_sum = 0
for token, freq in token_freqs.items():
    total_sum += freq

print(total_sum)

model = {}
for token, freq in token_freqs.items():
    probability = freq / total_sum
    negative_log_probability = -log(probability)
    model[token] = negative_log_probability

print(model)

594
{'▁': 2.952892114877499, 'T': 5.288267030694535, 'h': 4.189654742026425, 'i': 3.821929961901108, 's': 3.821929961901108, 't': 3.7478219897473863, 'e': 3.342356881639222, 'H': 5.6937321388027, 'u': 4.59511985013459, 'g': 4.777441406928545, 'n': 3.9889840465642745, 'F': 6.386879319362645, 'a': 3.9019726695746444, 'c': 5.288267030694535, 'C': 6.386879319362645, 'o': 3.821929961901108, 'r': 4.189654742026425, '.': 5.000584958242754, 'p': 5.6937321388027, 'b': 5.288267030694535, 'k': 5.288267030694535, 'z': 5.6937321388027, 'w': 5.288267030694535, 'v': 6.386879319362645, 'l': 4.440969170307332, 'm': 6.386879319362645, 'f': 6.386879319362645, 'y': 5.288267030694535, ',': 6.386879319362645, 'd': 5.000584958242754, '▁t': 4.440969170307332, 'is': 4.777441406928545, 'er': 4.777441406928545, '▁a': 4.777441406928545, '▁to': 5.000584958242754, 'to': 5.000584958242754, 'en': 5.000584958242754, '▁T': 5.288267030694535, '▁Th': 5.288267030694535, '▁Thi': 5.288267030694535, '▁This': 5.28826703069453

Now the main function is the one that tokenizes words using the Viterbi algorithm. As we saw before, that algorithm computes the best segmentation of each substring of the word, which we will store in a variable named best_segmentations. We will store one dictionary per position in the word (from 0 to its total length), with two keys: the index of the start of the last token in the best segmentation, and the score of the best segmentation. With the index of the start of the last token, we will be able to retrieve the full segmentation once the list is completely populated.

Populating the list is done with just two loops: the main loop goes over each start position, and the second loop tries all substrings beginning at that start position. If the substring is in the vocabulary, we have a new segmentation of the word up until that end position, which we compare to what is in best_segmentations.

Once the main loop is finished, we just start from the end and hop from one start position to the next, recording the tokens as we go, until we reach the start of the word:

In [9]:
def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)):
        # This should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                # If we have found a better segmentation ending at end_idx, we update
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        # We did not find a tokenization of the word -> unknown
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score

In [10]:
# We can already try our initial model on some words:

print(encode_word("Hopefully", model))
print(encode_word("This", model))

(['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y'], 41.5157494601402)
(['This'], 6.288267030694535)


In [11]:
# Now it’s easy to compute the loss of the model on the corpus!
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

# We can check it works on the model we have:
compute_loss(model)

413.10377642940875

In [12]:
# Computing the scores for each token is not very hard either; 
# we just have to compute the loss for the models obtained by deleting each token:

import copy

def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

# We can try it on a given token:
scores = compute_scores(model)
print(scores["ll"])
print(scores["his"])

6.376412403623874
0.0


Since "ll" is used in the tokenization of "Hopefully", and removing it will probably make us use the token "l" twice instead, we expect it will have a positive loss. "his" is only used inside the word "This", which is tokenized as itself, so we expect it to have a zero loss. Here are the results:

With all of this in place, the last thing we need to do is add the special tokens used by the model to the vocabulary, then loop until we have pruned enough tokens from the vocabulary to reach our desired size:

In [13]:
percent_to_remove = 0.1
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    # Remove percent_to_remove tokens with the lowest scores.
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])

    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

Then, to tokenize some text, we just need to apply the pre-tokenization and then use our encode_word() function:

In [14]:
def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])


tokenize("This is the Hugging Face course.", model)

['▁This',
 '▁is',
 '▁the',
 '▁Hugging',
 '▁Face',
 '▁',
 'c',
 'ou',
 'r',
 's',
 'e',
 '.']